In [5]:
# Parameters
artificial_humans = "../../data/training/ah_10/data/model.pt"
artificial_humans_model = "graph"
output_path = "../../data/manager/v2/dev/"
manager_args = {
    "opt_args": {"lr": 0.003},
    "gamma": 1.0,
    "eps": 0.2,
    "target_update_freq": 20,
    "model_args": {
        "hidden_size": 5,
        "add_rnn": True,
        "add_edge_model": True,
        "add_global_model": False,
        "x_encoding": [
            {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
            {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
        ],
        "u_encoding": [
            {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
            {"name": "prev_common_good", "norm": 128, "etype": "float"},
        ],
    },
}
replay_memory_args = {"n_episodes": 20, "sample_size": 5}
n_update_steps = 10
eval_args = {"test_period": 10, "batch_size": 1000}
env_args = {
    "n_agents": 4,
    "n_contributions": 21,
    "n_punishments": 31,
    "episode_steps": 16,
    "batch_size": 10,
}
device = "cpu"


In [6]:
%load_ext autoreload
%autoreload 2

import torch as th
import os
from itertools import count

from aimanager.manager.memory import Memory
from aimanager.manager.environment import ArtificialHumanEnv
from aimanager.artificial_humans import AH_MODELS
from aimanager.manager.manager import ArtificalManager
from aimanager.manager.graph_memory import GraphMemory
from aimanager.manager.evaluation import ManagerEvaluator

from torch_geometric.data import Batch


output_file =  os.path.join(output_path, 'data', "eval")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
state

NameError: name 'state' is not defined

In [13]:
device = th.device(device)
artifical_humans = AH_MODELS[artificial_humans_model].load(artificial_humans).to(device)

env = ArtificialHumanEnv(
    artifical_humans=artifical_humans, device=device, **env_args)

n_episode_steps = env.episode_steps

manager = ArtificalManager(
    n_contributions=env.n_contributions, n_punishments=env.n_punishments, 
    default_values=artifical_humans.default_values, device=device, **manager_args)

# replay_mem = GraphMemory(n_episode_steps=env.episode_steps, n_nodes=env.n_agents, device=device, **replay_memory_args)
# recorder = Memory(n_episodes=n_episodes, n_episode_steps=env.episode_steps, device=device)
# evaluator = ManagerEvaluator(n_update_steps, env.episode_steps, output_file=output_file, **eval_args)

for update_step in range(n_update_steps):

    state = env.init_episode()

    manager.init_episode(update_step)

    for step in count():
        state_ = {k: v.unsqueeze(-1) for k, v in state.items()}
        obs = Batch.from_data_list(manager.encode(state_, edge_index=env.edge_index))

        # Get q values from controller
        q_values = manager.get_q(manager_observations=obs, first=step == 0).squeeze(1)

        # Sample a action
        selected_action = manager.eps_greedy(q_values=q_values)

        state = env.punish(selected_action)
        recorder.add(**state, episode_step=step)

        # pass actions to environment and advance by one step
        state, reward, done = env.step()
        replay_mem.add(
            action=selected_action, reward=reward, 
            obs=obs)
 
        if done:
            replay_mem.next_episode(episode)
            
            # allow manager to update itself
            sample = replay_mem.sample(**sample_args)
            

            if sample is not None:
                manager.update(**sample)
            break
    # recorder.add(episode_step=step, **state)
    # recorder.next_episode(episode)
    evaluator.eval_episode(manager, env, episode)


torch.Size([40, 1])


IndexError: The shape of the mask [10, 4] at index 0 does not match the shape of the indexed tensor [40, 1] at index 0